# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [5]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [169]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """
    
    def __init__(self, location, gap=28, speed_limit=33, start_speed=28):
        self.location = location
        self.gap = gap
        
        # speed limit is in km/hr = 1000 m / 3600 s = 1/3.6 m/s;
        self.desired_speed = speed_limit
        
        self.speed = start_speed
        self.size = 5
        self.advance = None
    
    def __str__(self):
        return 'Car(location={},gap={})'.format(self.location, self.gap)
    
    def __repr__(self):
        return self.__str__()
    
    def drive(self, car_ahead): # TODO: UPDATE GAP!
        self.advance = self.speed - car_ahead.speed

        if car_ahead.location - (car_ahead.size - 1) - self.location > self.location - car_ahead.location:
            self.gap = car_ahead.location - (car_ahead.size - 1) - self.location
        else:
            self.gap = car_ahead.location + 1000 - (car_ahead.size - 1) - self.location

        if self.stop(car_ahead):
            return self
        elif self.match_speed(car_ahead):
            return self
        elif self.random_slowdown():
            return self
        elif self.accelerate():
            return self

    def stop(self, car_ahead):
        if self.gap - self.advance <= 0:
            self.speed = 0
            self.location = car_ahead.location - (car_ahead.size - 1)
            return True
        
    def match_speed(self, car_ahead):
        if self.gap - self.advance <= self.speed:
            self.speed = car_ahead.speed
            return True
        
    def random_slowdown(self):
        if random.random() < 0.1:
            self.speed -= 2
            return True
    
    def accelerate(self):
        if self.speed < self.desired_speed:
            self.speed += 2
            return True

In [212]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - (1000 - sum(vehicle.size)) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self, num_trucks=0):
        self.total_vehicle_space = ((30 - num_trucks) * 5) + (num_trucks * 25)
        self.initial_gap = (1000 - self.total_vehicle_space) // 30
        self.vehicles = [Car((4 + 33*n), self.initial_gap) for n in range(30 - num_trucks)]
        
        if num_trucks > 0:
            [self.vehicles.append(Truck() for _ in range(num_trucks))]

        self.vehicles[-1].gap = (1000 - self.vehicles[-1].location)
        
    def reinsert_car(self):
        if self.vehicles[-1].location < self.total_vehicle_space:
            self.vehicles.insert(0, self.vehicles.pop())
            return True

In [214]:
class HighwaySim:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
        - Update locations; location - 1000 if > 1000
    - Report stats; return traffic jam status
    - Pop, Append
    
    Collaborators:
    - Car
    - Road
    """
    
    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.is_traffic = []
    
    def iterate(self):
        """
        Reverse list to process cars at the end of the road first.
        n.b., lower numbers are physically ahead of higher numbers.
        """
        for idx, v in enumerate(self.road.vehicles[::-1]):
            v.drive(self.road.vehicles[idx - 1])
            
            if v.location + v.speed < 1000:
                v.location = v.location + v.speed
            else:
                v.location = v.location + v.speed - 1000
                self.road.reinsert_car()
            
            if v.speed == 0:
                self.is_traffic.append(True)
            else:
                self.is_traffic.append(False)
    
    def run_sim(self, duration=2):
        while self.ticks < duration:
            self.iterate()
            self.ticks += 1
        if self.is_traffic.count(True) > 0:
            return True
        else:
            return False

In [215]:
sim1 = HighwaySim()

In [216]:
sim1.run_sim()

False

In [217]:
sim1.road.vehicles

[Car(location=36,gap=996),
 Car(location=85,gap=961),
 Car(location=128,gap=885),
 Car(location=161,gap=819),
 Car(location=194,gap=753),
 Car(location=227,gap=687),
 Car(location=260,gap=621),
 Car(location=293,gap=555),
 Car(location=326,gap=489),
 Car(location=351,gap=423),
 Car(location=392,gap=357),
 Car(location=425,gap=291),
 Car(location=458,gap=225),
 Car(location=491,gap=159),
 Car(location=524,gap=85),
 Car(location=549,gap=963),
 Car(location=590,gap=897),
 Car(location=623,gap=831),
 Car(location=656,gap=765),
 Car(location=689,gap=699),
 Car(location=722,gap=633),
 Car(location=755,gap=567),
 Car(location=788,gap=501),
 Car(location=821,gap=435),
 Car(location=854,gap=369),
 Car(location=887,gap=303),
 Car(location=920,gap=237),
 Car(location=953,gap=171),
 Car(location=986,gap=95),
 Car(location=3,gap=70)]

In [147]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        trial_data = HighwaySim()
        trial_stats.append(trial_data)
    return trial_stats

In [148]:
highway_1000_data = highway_trials()

In [149]:
print(highway_1000_data.count(True))

0
